<a href="https://colab.research.google.com/github/dcsan/dalle-playground/blob/main/backend/dalle_playground_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dc fork v12

<center><img src="https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/240/apple/285/woman-artist_1f469-200d-1f3a8.png" width="120">
</center>

### <center>Use this notebook to run your DALL-E server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>

#####<center>This notebook is based on the amazing [DALL·E Mini](https://github.com/borisdayma/dalle-mini) by [Boris Dayma](https://github.com/borisdayma) et al.</center>

## Install dependencies

In [1]:

!ls
![ ! -d 'dalle-playground' ] && git clone https://github.com/dcsan/dalle-playground.git && pip install -r dalle-playground/backend/requirements.txt
!pip install pyngrok

# only if we need to reset:
# !rm -rf dalle-playground
# !git clone https://github.com/dcsan/dalle-playground.git
# !pip install -r dalle-playground/backend/requirements.txt
# !npm install -g localtunnel


sample_data
Cloning into 'dalle-playground'...
remote: Enumerating objects: 673, done.
remote: Counting objects: 100% (673/673), done.
remote: Compressing objects: 100% (280/280), done.
remote: Total 673 (delta 417), reused 598 (delta 391), pack-reused 0
Receiving objects: 100% (673/673), 1.55 MiB | 3.44 MiB/s, done.
Resolving deltas: 100% (417/417), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ymxkclqp
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-ymxkclqp
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/patil-suraj/vqgan-jax.git to /tmp/pip-req-build-499ks1sf
  Running command git clone -q https://github.com/patil-suraj/vqgan-jax.git /tmp/pip-req-build-499ks1sf
  Cloning https://github.com/b

## Choose a model

* **DALL-E Mini**
  * The original DALL-E Mini. Fastest yet suboptimal results
  * Runs well on a 4GB GPU and the Google Colab free tier
* **DALL-E Mega**
  * The advanced version of DALL-E Mini. Requires more compute and VRAM
  * Runs well on a 8GB GPU and recommended on the Google Colab Pro tier
* **DALL-E Mega Full**
  * The most performant DALL-E version. Requires significantly more computre and VRAM
  * Runs well on a 12GB GPU and recommended on the Google Colab Pro+ tier


In [2]:
# # get/install ngrok
# !wget --no-check-certificate -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !wget --no-check-certificate -nc https://eyalgruss.com/fomm/ngrok-stable-linux-amd64.zip
# !unzip -n ngrok-stable-linux-amd64.zip
# !pkill -f ngrok
# !nohup /content/ngrok http -region us -inspect=false $port &
# !npm install -g localtunnel

# !npm install -g npm

# check we have a GPU
!nvidia-smi


Sat Jul 23 23:46:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#@title Select a model - remember to execute cell after selecting!

dalle_model = 'Mega' #@param ["Mini", "Mega", "Mega_full"]

# Start the tunnel

In [8]:
from threading import Thread
from pyngrok import ngrok
import time
import sys
import requests
app_port = 8000
update_url = 'http://revel.alfalabs.xyz/api/revel/imagenServer/set'
ngrok.set_auth_token("2CJlmTjS9dlNZJEhhrNsrKsgZGj_bHfQrwsRE4AxLodRWmGa")
# 2BGQ9byQL3Cjbfo7d2buDy7cgz9_3NMQJGVGQhcQZDptuXQND


def stop_ngrok():
  print('stopping ngrok')
  tunnels = ngrok.get_tunnels()
  for tunnel in tunnels:
    print('closing tunnel', tunnel.public_url)
    ngrok.disconnect(tunnel.public_url)

def restart_ngrok():
  # close existing bg processes
  # free token
  stop_ngrok()
  ngcon = ngrok.connect(app_port)
  puburl = ngcon.public_url
  print('ngrok public_url\n', puburl)
  ngrok_process = ngrok.get_ngrok_process()
  start = time.time()
  json = {'url': puburl}
  result = requests.post(update_url, json = json)
  print(result.status_code, result.json())

  try:
    # Block until CTRL-C or some other terminating event
    # time.sleep(10)
    ngrok_process.proc.wait()
  except KeyboardInterrupt:
    print(" Shutting down server")
    ngrok.kill()
  finally:
    now = time.time()
    uptime = round(now - start)
    print(f'uptime {uptime}s')
    print(f'uptime {round(100 * uptime/60) / 100} mins')

# restart_ngrok() # test
stop_ngrok()

stopping ngrok


In [ ]:
def app():
  print(f"Selected DALL-E Model - [{dalle_model}]")
  !python dalle-playground/backend/app.py --port {app_port} --model_version {dalle_model} --save_to_disk true --img_format jpeg --output_dir generations
  print(f'dalle starting on {app_port}')


# def warmup():
#   check = requests.get('http://127.0.0.1:8000')
#   print('check', check)
#   try:
#     # Block until CTRL-C or some other terminating event
#     time.sleep(10)
#     # ngrok_process.proc.wait()
#   except KeyboardInterrupt:
#     print(" Shutting down server")
#     ngrok.kill()
#   finally:
#     print('complete')

if __name__=='__main__':
  print('start thread')
  t1 = Thread(target = app)
  a = t1.start()
  # warmup()
  # !lt --port {app_port}
  restart_ngrok()
  # app()
  # warmup()

else:
  print('not main')


start thread
Selected DALL-E Model - [Mega]stopping ngrok

ngrok public_url
 http://6da2-34-80-201-136.ngrok.io
--> Starting DALL-E Server. This might take up to two minutes.
200 {'ok': True, 'msg': 'updating AiGen server to: `[http://6da2-34-80-201-136.ngrok.io]`'}


In [ ]:
print("HI")

## Now use the updated URL
for the main application

it should look like this:

```
ngrok public_url
 http://xxxx-12-34-567-89.ngrok.io
```

## 🎉 Let the fun begin ✨
